In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
#driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install()))

In [24]:
import argparse
from enum import Enum
import sys
import time
import random
import pandas as pd
import numpy as np
import pathlib

In [25]:
def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def elementsBy(query,driver,**kargs):
    """Devuelve una lista de elements dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        List<Element>: Lista de elementos
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_elements(kargs.get("byq",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element


def get_data(driver,nombre):
    base="//div[@class='wanted_top_right']"
    wantedBy = elementBy(f"{base}/div[4]",driver,idLoader="//img",by=By.XPATH).text.replace("by ","")
    date = get_element(base,"//*[contains(text(),'Date of birth')]/../div[2]",driver)
    nacionalidad =  get_element(base,"//*[contains(text(),'Nationality')]/../div[2]",driver)
    etnia =  get_element(base,"//*[contains(text(),'Ethnic')]/../div[2]",driver)
    idiomas_hablados =  get_element(base,"//*[contains(text(),'Spoken ')]/../div[2]",driver)
    df=pd.DataFrame(columns=["Nombre","Perseguido","Fecha","Nacionalidad","Etnia","Idiomas"])
    df['Nombre']=[nombre]
    df['Perseguido']=[wantedBy]
    df['Fecha']=[date]
    df['Nacionalidad']=[nacionalidad]
    df['Etnia']=[etnia]
    df['Idiomas']=[idiomas_hablados]
    df.set_index("Nombre",inplace=True)
    return df

def get_element(base,query,driver):
    try:
        return elementBy(f"{base}{query}",driver,by=By.XPATH).text
    except:
        return np.nan

In [26]:
URL_BASE="https://eumostwanted.eu"

In [27]:
driver.get(URL_BASE)

In [28]:
buscados=elementsBy("//div[@class='wanted_list']/div",driver,needsWait=True,by=By.CLASS_NAME,idLoader="wanted_list")
df=pd.DataFrame([])
for buscado in buscados:
    nombre=buscado.get_dom_attribute("title")
    if nombre:
        array=nombre.lower().split(",")
        nombre=" ".join(array)
        if buscado.get_dom_attribute('href'):
            driver.execute_script(f"window.open('{URL_BASE}/#{buscado.get_dom_attribute('href')}')")
            driver.switch_to.window(driver.window_handles[1])
            df=pd.concat([df,get_data(driver,nombre)])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
df

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:662:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1473:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [ ]:
path=str(pathlib.Path().resolve())+"/"
df.to_parquet(path+"datos.parquet")

In [ ]:
df.groupby("Perseguido")['Perseguido'].value_counts()

Perseguido
                  28
Austria            1
Belgium            1
Bulgaria           2
Croatia            1
Czech Republic     2
Denmark            1
France             3
Germany            2
Hungary            1
Italy              1
Latvia             2
Luxembourg         2
Malta              1
Poland             1
Romania            2
Slovakia           2
Spain              2
Sweden             1
Name: count, dtype: int64

In [ ]:
df['Edad'] = df['Fecha'].str.split("(").str[1].str.split(" ").str[0].fillna(0).astype(np.int64)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, garweg  burkhard to leijdekkers  joseph johannes (jos)
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Perseguido    56 non-null     object
 1   Fecha         55 non-null     object
 2   Nacionalidad  53 non-null     object
 3   Etnia         38 non-null     object
 4   Idiomas       41 non-null     object
 5   Edad          56 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 5.1+ KB


In [ ]:
print(f"Media:{df['Edad'].mean()}")
print(f"Mediana:{df['Edad'].median()}")
Q3 = df['Edad'].quantile(0.75)
Q1 = df['Edad'].quantile(0.25)
print("Personas que superan valores extremos Quantile 1/3:")
df[(df['Edad']>Q3) | (df['Edad']<Q1)]

Media:44.05357142857143
Mediana:44.5
Personas que superan valores extremos Quantile 1/3:


,Perseguido,Fecha,Nacionalidad,Etnia,Idiomas,Edad
Nombre,,,,,,
garweg burkhard,Germany,"September 1, 1968 (55 years)",German,European,German,55
staub ernst-volker,Germany,"October 30, 1954 (69 years)",German,European,German,69
calleja maatouk marzia marimar,,"April 3, 2000 (23 years)",Maltese,European,English Maltese,23
gomez tania,,"July 24, 1992 (31 years)",Swedish,European,Spanish Swedish,31
cinquegranella renato,Italy,"May 15, 1949 (74 years)",Italian,European,Italian,74
haredin fejzulla,,"June 5, 1965 (58 years)",Albanian,NaN,NaN,58
de la camara guisado ramón,Spain,"August 29, 1952 (71 years)",Spanish,European,Spanish,71
burreli aleks,,NaN,NaN,NaN,NaN,0
motisi giovanni,,"January 1, 1959 (65 years)",NaN,European,Italian,65
